### Downloads Files

In [ ]:
#-*- encoding: gbk -*-
# python3
import pandas as pd
import smtplib,imaplib
import email,time,os,pdb,sys,datetime
from email.header import decode_header

import importlib
importlib.reload(sys)

class Package_emails(object):
    
    def __init__(self,endDate, dte, account, password, mailhost, dir1, exts=['.xls', '.xlsx']):
        self.endDate  = endDate
        self.dte      = dte
        self.account  = account
        self.password = password
        self.mailhost = mailhost
        self.path     = dir1
        self.exts     = exts

        
    def _get_attachments(self, msg):
        
        attachments = []
        for part in msg.walk():
            if part.get_content_maintype() == 'multipart':
                continue
            if part.get('Content-Disposition') is None:
                continue
            
            # 获取附件文件名
            fileName = part.get_filename()
            fileName = decode_header(fileName)[0][0].decode(decode_header(fileName)[0][1])
            
            # 只获取指定拓展名的附件
            extension = os.path.splitext(os.path.split(fileName)[1])[1]
            if extension not in self.exts:
                continue
            
            print('Downloading ', fileName)
            # 如果获取到了文件，则将文件保存在指定的目录下
            if fileName:
                filePath = os.path.join(self.path, fileName)
                fp       = open(filePath, 'wb')
                fp.write(part.get_payload(decode=True))
                fp.close()
                attachments.append(fileName)

        return attachments
            
            
    #获取邮件方法
    def _connect_mail(self,port = 993):
        
            print('Begin Connecting ...')
            #连接到估值表所在文件夹
            conn = imaplib.IMAP4_SSL(self.mailhost, port)
            conn.login(self.account, self.password)
            conn.select("INBOX")                      #收件夹为例
            print('Login Successfully !')
            
            #提取文件夹下所有邮件的编号
            resp, mails = conn.search(None, "ALL")
            msgList = mails[0].split()
            
            print('Begin Scanning ...')
            # 从最近的邮件开始获取
            for i in reversed(range(len(msgList))):
                print(i)
                try:
                    resp, data = conn.fetch(msgList[i], '(RFC822)')
                    emailbody = data[0][1]
                    
                    mail = email.message_from_bytes(emailbody)
                    
                    # 获取邮件主题
                    subject = mail['Subject']
                    subdecode = decode_header(subject)
                    sender = mail['From']
                    
                    if subdecode[0][1] == None:
                        title = subdecode[0][0]
                    else:
                        title = subdecode[0][0].decode(subdecode[0][1])
                    
                    # 筛选特定主题
                    endDte1 = str(self.endDate)
                    endDte2 = endDte1[0:4] + '-' + endDte1[4:6] + '-' + endDte1[6:8]
                    if 'xxx' in title and ((endDte1 in title) or (endDte2 in title)):  #words in subject if needed
                        print(title)
                        # 获取附件
                        attachments = self._get_attachments(mail)
                except Exception as e:
                    print('ERROR:\n subject-> ',subject, '\n send by-> ',sender,'\n subdecode-> ',subdecode)  # 有极个别的可能会解码不了，可去邮箱手动下载
                    continue
            
            print('All Done !!!')
            
            
            conn.close()    
            conn.logout() 
            
            
            
if __name__ =="__main__":
    
    dte = int(datetime.datetime.today().strftime('%Y%m%d'))
    endDate = 20210930        #date in subject if needed
    
    mail_accounts = {
        'default':{
            'imapServerADD' : "imap.qiye.163.com",  #your email engine
            'emailUsr' : "xxx@xxx",                 #your email address
            'emailPwd' : "",                        #your pw
            'tag' : None,
        },
    }
    
    dir1 = ' ' #your path
    if not os.path.exists(dir1):
        os.makedirs(dir1)
    
    account  = mail_accounts['default']['emailUsr']
    password = mail_accounts['default']['emailPwd']
    mailhost = mail_accounts['default']['imapServerADD']
    Package_emails(endDate, dte, account, password, mailhost, dir1)._connect_mail()
    
    